# Determinare le scale adatte per il multislide

## Idea di base
L'idea di base è quella di prendere le bounding box delle immagini, quindi determinare la bounding box più piccola e quella più grande (in termini di aree).

Dopodiché cercare una scala adatta affinché moltiplicando la dimensione della finestra raggiunge quella desiderata

In [3]:
# import delle librerie
import cv2
import numpy as np

In [4]:
def assign_sets():
    '''
    Questa funzione legge dalla cartella i seguenti file:
    - test_assignment.txt
    - train_assignment.txt
    - val_assignment.txt
    Poi ritorna tre array di valori
    '''
    test_set = []
    train_set = []
    val_set = []
    # leggo prima Train
    with open('train_assignment.txt', 'r') as f:
        for row in f:
            row = row.strip()
            train_set.append(row)
    # poi test
    with open('test_assignment.txt', 'r') as f:
        for row in f:
            row = row.strip()
            test_set.append(row)
    # infine validation
    with open('val_assignment.txt', 'r') as f:
        for row in f:
            row = row.strip()
            val_set.append(row)
    
    print(f'Train set: {len(train_set)}')
    print(f'Test set: {len(test_set)}')
    print(f'Val set: {len(val_set)}')
    return train_set, test_set, val_set

In [5]:
def extract_boundary_box(valore, annotations):
    '''
    Questa funzione prende un valore in input, ovvero l'immagine dove cercare
    ed estrae le boundary box della classe 1 relative a quel valore.
    '''
    path_file = 'Annotations/' + valore + '.jpg.txt'
    # print(path_file)
    rows = []
    if path_file[12:] in annotations:
        # il file è presente in Annotations
        with open(path_file, 'r') as f:
            for row in f:
                row = row.strip()
                rows.append(row)
        rows = rows[1:] # faccio slicings
        bboxes = []
        for row in rows:
            row = row.split(' ')
            target,x,y,w,h = row
            if int(target) == 1:
                box = [x,y,w,h]
                bboxes.append(box)
        # print(f'Ho trovato {len(bboxes)} boundary boxes')
        return bboxes
    return None

In [6]:
import os
directory = 'Annotations'
annotations = []
for root,dirs,files in os.walk(directory):
    for file in files:
        annotations.append(str(file))

In [7]:
# creo i set
train, test, val = assign_sets()

bboxes_train = []
for i in range(len(train)):
    bboxes = extract_boundary_box(train[i], annotations)
    if bboxes:
        # aggiungo alle Boundary Box anche il nome dell'immagine dove andare a cercare
        bboxes_train.append((train[i], bboxes))

bboxes_test = []
for i in range(len(test)):
    bboxes = extract_boundary_box(test[i], annotations)
    if bboxes:
        # aggiungo alle Boundary Box anche il nome dell'immagine dove andare a cercare
        bboxes_test.append([test[i], bboxes])

bboxes_val = []
for i in range(len(val)):
    bboxes = extract_boundary_box(val[i], annotations)
    if bboxes:
        # aggiungo alle Boundary Box anche il nome dell'immagine dove andare a cercare
        bboxes_val.append([val[i], bboxes])


print(f'Ci sono in totale {len(bboxes_train)} file di annotazioni per il train')
print(f'Ci sono in totale {len(bboxes_test)} file di annotazioni per il test')
print(f'Ci sono in totale {len(bboxes_val)} file di annotazioni per il validation')

Train set: 7000
Test set: 1000
Val set: 1000


Ci sono in totale 6919 file di annotazioni per il train
Ci sono in totale 983 file di annotazioni per il test
Ci sono in totale 991 file di annotazioni per il validation


In [8]:
example_img_bboxes = bboxes_train[56]
example_img_bboxes

('000408',
 [['86', '210', '116', '287'],
  ['195', '227', '233', '320'],
  ['288', '206', '318', '279'],
  ['338', '203', '359', '255'],
  ['372', '202', '389', '243'],
  ['394', '200', '409', '238'],
  ['442', '211', '479', '303'],
  ['408', '200', '418', '228'],
  ['420', '198', '431', '224'],
  ['111', '200', '132', '254'],
  ['25', '198', '43', '241'],
  ['0', '211', '27', '281'],
  ['65', '204', '81', '244'],
  ['50', '196', '63', '228'],
  ['181', '208', '194', '243'],
  ['197', '212', '215', '255'],
  ['130', '201', '146', '241'],
  ['256', '211', '271', '247'],
  ['269', '197', '282', '231'],
  ['228', '201', '241', '235'],
  ['26', '198', '43', '241'],
  ['329', '195', '343', '229']])

In [9]:
len(example_img_bboxes[1])

22

In [10]:
example_img_bboxes[1][0]

['86', '210', '116', '287']

- prendi ogni bbox
- calcola l'area
- salva l'area in un vettore il dizionario {bbox:area}
- ordina le bbox per area
- estrai massimo e minimo
- per massimo e minimo trova la scala f (con f che varia da 1 a 0.1 con passo 0.1) affinché la nuova area della winSize calcolata come w'=w/f, h'=h/f sia "vicina" a quella del massimo e del minimo. Poi fai lo stesso con un terzo valore intermedio

In [19]:
def find_factor_scale(boundary, winSize):
    # 408, 200, 418, 228
    delta_x_1 = boundary[2] - boundary[0]
    best_f = 1
    areas = []
    for i in range(10, 1, 1):
        i = i/10
        f = i
        best_f = f
        w_ = winSize[1] / f
        h_ = winSize[0] / f
        if (delta_x_1 / (f*w_)) <=  1:
            break
    return best_f

def find____factor_scale(boundary, winSize):
    area_size = winSize[0] * winSize[1]
    b_area = boundary[1]
    best_f = 1
    areas = []
    for i in range(1, 11, 1):
        i = i/10
        f = i
        w_ = winSize[1] / f
        h_ = winSize[0] / f
        new_area = w_ * h_
        if new_area >= b_area:
            areas.append((new_area, f))
    areas.sort(key=lambda x: x[1])
    best_f = areas[-1][1]
    return best_f

In [20]:
def find_scale(bboxes, winSize=(64,128)):
    bboxes_area = []
    for bbox in bboxes:
        # faccio il casting degli interi
        for i in range(len(bbox)):
            bbox[i] = int(bbox[i])
        x1,y1,x2,y2 = bbox
        area = abs((x2-x1)*(y2-y1))
        bbox_area = (bbox,area)
        bboxes_area.append(bbox_area)
    # print(bboxes_area)
    bboxes_area.sort(key=lambda tup: tup[1])
    # print(bboxes_area)
    min_boundary = bboxes_area[0]
    max_bounday = bboxes_area[-1]
    # medium = len(bboxes_area) // 2
    # med_boundary = bboxes_area[medium]
    print(min_boundary)
    print(max_bounday)
    # print(med_boundary)
    f_max = find_factor_scale(max_bounday[0], winSize)
    f_min = find_factor_scale(min_boundary[0], winSize)
    f_med = (f_max + f_min) / 2
    print(f'F max: {f_max}')
    print(f'F min: {f_min}')
    # print(f'F med: {f_med}')
    return f_max, f_med, f_min

        

In [21]:
find_scale(example_img_bboxes[1])

([408, 200, 418, 228], 280)
([195, 227, 233, 320], 3534)
F max: 1
F min: 1


(1, 1.0, 1)